In [1]:
import numpy as np
import pandas as pd
import pickle
from keras.models import Sequential, Model
from keras.layers import Input, Dense, LSTM, Dropout, Activation, Permute, GRU, Reshape
from keras.layers import MaxPooling2D, Flatten, Conv2D, BatchNormalization, Lambda, Bidirectional, concatenate
from keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from keras import regularizers
from keras.optimizers import RMSprop
import keras.backend as K
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
import keras.utils
from keras.models import load_model
from tensorflow import set_random_seed

K.set_image_data_format('channels_last')
set_random_seed(24)
np.random.seed(25)

Using TensorFlow backend.


In [2]:
spec_path = '../input/spect-scaled/all_spects_scaled.npz'
with open(spec_path, 'rb') as handle:
    all_spects = np.load(handle)['arr_0']

info_path = '../input/track-info/track_info.pickle'
with open(info_path, 'rb') as handle:
    track_info = pickle.load(handle)

track_ids = track_info.index
spect_track_id_dict = dict(zip(track_ids, all_spects))

enc = LabelEncoder()
track_info['labels'] = enc.fit_transform(track_info['genre_top'])
y_cat = keras.utils.to_categorical(track_info['labels'].values)
track_info['one_hot_labels'] = [y.tolist() for y in y_cat]

In [3]:
def make_dataset_splits(split):
    """Return spectrogram arrays for the given dataset split"""
    mask = track_info.split == split
    ids = track_info.loc[mask].index
    data = np.asarray([spect_track_id_dict[i] for i in ids])
    data = data.reshape(data.shape[0], data.shape[1], data.shape[2], 1)
    return data


def get_input_data(split):

    assert (split == 'training') or (split == 'validation') or (split == 'test') or (split == 'all')

    if split == 'training':
        x_train = make_dataset_splits('training')
        return x_train
    elif split == 'validation':
        x_val = make_dataset_splits('validation')
        return x_val
    elif split == 'test':
        x_test = make_dataset_splits('test')
        return x_test
    else:
        x_train = make_dataset_splits('training')
        x_val = make_dataset_splits('validation')
        x_test = make_dataset_splits('test')
        return x_train, x_val, x_test


def get_input_shape():

    shape = get_input_data(split='test')[0].shape
    return shape


def make_label_splits(split, label_type = 'one_hot_labels'):
    """Return response labels for the given dataset split"""
    mask = track_info.split == split
    labels = track_info.loc[mask, label_type]
    labels = np.asarray([np.array(l) for l in labels])

    return labels


def get_labels(split, label_type = 'one_hot_labels'):
    assert (split == 'training') or (split == 'validation') or (split == 'test') or (split == 'all')

    if split == 'training':
        y_train = make_label_splits('training',label_type)
        return y_train
    elif split == 'validation':
        y_val = make_label_splits('validation',label_type)
        return y_val
    elif split == 'test':
        y_test = make_label_splits('test',label_type)
        return y_test
    else:
        y_train = make_label_splits('training', label_type)
        y_val = make_label_splits('validation', label_type)
        y_test = make_label_splits('test', label_type)
        return y_train, y_val, y_test


def get_output_shape():
    shape = get_labels(split='test')[0].shape
    return shape

In [4]:
def cnn(param_grid):
    """Method for creating a CNN Keras model object with the parameters given by the the param_grid"""
    n_filters = param_grid['n_filters']
    filter_size = param_grid['filter_size']
    pool_size = param_grid['pool_size']
    dropout_rate = param_grid['dropout_rate']
    n_cov_layers = param_grid['n_cov_layers']
    n_dense_nodes = param_grid['n_dense_nodes']
    input_shape = param_grid['input_shape']

    model = Sequential()

    for i in range(n_cov_layers):
        model.add(Conv2D(filters=n_filters, kernel_size=filter_size, activation='relu',
                         input_shape=(input_shape[-3], input_shape[-2], 1), padding='same',
                         data_format='channels_last', name='conv2d_' + str(i)))
        model.add(BatchNormalization(name='batch_norm_' + str(i)))
        model.add(Activation('relu', name='relu_' + str(i)))
        model.add(MaxPooling2D(pool_size=pool_size, data_format="channels_last", name='max_pool2d_' + str(i)))
        model.add(Dropout(dropout_rate, name='dropout_' + str(i)))

    model.add(Flatten(name='flatten'))

    model.add(Dense(n_dense_nodes, activation='relu', name='dense_1'))
    model.add(Dense(n_dense_nodes, activation='relu', name='dense_2'))
    model.add(Dense(8, activation='softmax', name='output'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [5]:
def crnn_parallel(param_grid):
    """Method for creating a CRNN Keras model object with the parameters given by the the param_grid"""
    n_filters = param_grid['n_filters']
    filter_size = param_grid['filter_size']
    pool_size = param_grid['pool_size']
    dropout_rate = param_grid['dropout_rate']
    n_cov_layers = param_grid['n_cov_layers']
    input_shape = param_grid['input_shape']

    input_layer = Input(shape=(input_shape[-3], input_shape[-2], input_shape[-1]),
                        name='input')
    layer = input_layer
    for i in range(n_cov_layers):
        layer = Conv2D(filters=n_filters[i], kernel_size=filter_size, padding='valid',
                       strides=1, name='conv2d_' + str(i))(layer)
        layer = BatchNormalization(axis = 3, name = 'batch_norm' + str(i))(layer)
        layer = Activation('relu', name = 'relu' + str(i))(layer)
        layer = MaxPooling2D(pool_size=pool_size[i], name='pool2d' + str(i))(layer)
        layer = Dropout(dropout_rate, name = 'dropout' + str(i))(layer)

    layer = Flatten()(layer)

    r_layer = MaxPooling2D(pool_size[5], name='pool_lstm')(input_layer)
    r_layer = Lambda(lambda x: K.squeeze(x, axis=-1))(r_layer)

    r_layer = Bidirectional(GRU(64), merge_mode='concat')(r_layer)

    concat = concatenate([layer, r_layer], axis=-1, name='concat')

    output_layer = Dense(8, activation='softmax', name='output')(concat)

    opt = RMSprop(lr=0.0005)
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [6]:
def train_model(model,model_name, epochs=20, batch_size=64):
    """
     - Method for training a given Keras model with the given number of training epochs and batch size.
     - Methods saves the weights of the best model to a predetermined directory.
     - Method includes ModelCheckpoint, ReduceLROnPlateau and TensorBoard callbacks.
     - Method returns fitted model and model history"""

    x_train = get_input_data('training')
    x_val = get_input_data('validation')

    y_train = get_labels('training')
    y_val = get_labels('validation')

    # Edit path to desired desitation
    best_model_path = './models/' + model_name + '/weights.best.h5'
    best_model = ModelCheckpoint(best_model_path, monitor='val_acc',
                                 save_best_only=True, mode='max')
    rp_callback = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=10, min_delta=0.0)

    tensorboard_path = './models/' + model_name + '/logs'
    tensorboard = TensorBoard(log_dir=tensorboard_path)

    hist = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val),
                     callbacks=[best_model, rp_callback, tensorboard])
    return model, hist


def train_cnn():
    """Train CNN model with validation set preset parameters. Returns the fitted model and training history"""
    param_grid = {'n_filters': 60, 'filter_size': (5, 5), 'pool_size': (2, 2),
                  'input_shape': get_input_shape(),
                  'dropout_rate': 0.3, 'n_cov_layers': 5, 'n_dense_nodes': 240}

    cnn1 = cnn(param_grid)
    cnn_fitted, cnn_history = train_model(cnn1, model_name='cnn1')

    return cnn_fitted, cnn_history


def train_crnn_parallel():
    """Train CRNN model with validation set preset parameters. Returns the fitted model and training history"""
    crnn_parallel_param_grid = dict(n_filters=[16, 32, 64, 64, 64], filter_size=(3, 1),
                                    pool_size=[(2, 2), (2, 2), (2, 2), (4, 4), (4, 4), (4, 2)], dropout_rate=0.3,
                                    n_cov_layers=5, n_recurrent_layers=2, n_time_layers=1, n_time_dst_nodes=128,
                                    n_dense_nodes=240, l2=.001, input_shape=get_input_shape(),
                                    output_shape=get_output_shape())
    crnn_parallel1 = crnn_parallel(crnn_parallel_param_grid)
    crnn_parallel_fitted , crnn_parallel_history = train_model(crnn_parallel1, model_name = 'crnn_parallel1', epochs = 50)

    return crnn_parallel_fitted, crnn_parallel_history


def predict_model(model, model_name):
    """
    Predict the training, validation, and testing accurcary for the given keras model.
    Returns pandas dataframes of the predictions for training, validation, and testing datasets
    Returns pandas series of the macro f1 scores for each dataset split """

    x_train, x_val, x_test = get_input_data('all')
    y_train, y_val, y_test = get_labels('all')

    train_loss, train_acc = model.evaluate(x_train, y_train)
    val_loss, val_acc = model.evaluate(x_val, y_val)
    test_loss, test_acc = model.evaluate(x_test, y_test)

    print('Training set Accuracy: ', train_acc)
    print('Validation set Accuracy: ', val_acc)
    print('Test set Accuracy: ', test_acc)

    train_predictions = model.predict(x_train)
    val_predictions = model.predict(x_val)
    test_predictions = model.predict(x_test)

    train_pred_classes = train_predictions.argmax(axis=-1)
    val_pred_classes = val_predictions.argmax(axis=-1)
    test_pred_classes = test_predictions.argmax(axis=-1)
    
    y_train, y_val, y_test = get_labels('all', label_type = 'labels')

    train_f1 = f1_score(y_train, train_pred_classes, average = 'macro')
    val_f1 = f1_score(y_val, val_pred_classes, average='macro')
    test_f1 = f1_score(y_test, test_pred_classes, average='macro')

    f1_scores = pd.Series({'Training': train_f1, 'Validation': val_f1, 'Testing': test_f1})
    f1_scores.name = model_name

    train_predictions_df = pd.DataFrame(train_predictions, columns=[l + '_prob' for l in enc.classes_],
                                        index=track_info.loc[track_info.split == 'training'].index)
    val_predictions_df = pd.DataFrame(val_predictions, columns=[l + '_prob' for l in enc.classes_],
                                      index=track_info.loc[track_info.split == 'validation'].index)
    test_predictions_df = pd.DataFrame(test_predictions, columns=[l + '_prob' for l in enc.classes_],
                                       index=track_info.loc[track_info.split == 'test'].index)

    return train_predictions_df, val_predictions_df, test_predictions_df, f1_scores

In [7]:
cnn_fitted, cnn_history = train_cnn()

Train on 6394 samples, validate on 800 samples
Epoch 1/20
6394/6394 [==============================] - 35s 6ms/step - loss: 2.1138 - acc: 0.2789 - val_loss: 2.0317 - val_acc: 0.1575
Epoch 2/20
6394/6394 [==============================] - 30s 5ms/step - loss: 1.6587 - acc: 0.3863 - val_loss: 2.0348 - val_acc: 0.1900
Epoch 3/20
6394/6394 [==============================] - 31s 5ms/step - loss: 1.5677 - acc: 0.4346 - val_loss: 12.2207 - val_acc: 0.1237
Epoch 4/20
6394/6394 [==============================] - 31s 5ms/step - loss: 1.4979 - acc: 0.4690 - val_loss: 2.0090 - val_acc: 0.3262
Epoch 5/20
6394/6394 [==============================] - 31s 5ms/step - loss: 1.4347 - acc: 0.4916 - val_loss: 1.7868 - val_acc: 0.3775
Epoch 6/20
6394/6394 [==============================] - 31s 5ms/step - loss: 1.3853 - acc: 0.5106 - val_loss: 1.8265 - val_acc: 0.3212
Epoch 7/20
6394/6394 [==============================] - 31s 5ms/step - loss: 1.3358 - acc: 0.5246 - val_loss: 1.8580 - val_acc: 0.3450
Epoch 8

In [8]:
train_predictions_df, val_predictions_df, test_predictions_df, cnn_f1_scores = predict_model(cnn_fitted, 'CNN')

800/800 [==============================] - 1s 2ms/step
Training set Accuracy:  0.4879574601281834
Validation set Accuracy:  0.42375
Test set Accuracy:  0.35375


In [9]:
cnn_f1_scores

Training      0.489975
Validation    0.420764
Testing       0.343055
Name: CNN, dtype: float64

In [10]:
#crnn_parallel_param_grid = dict(n_filters=[16, 32, 64, 64, 64], filter_size=(3, 1),
                                    #pool_size=[(2, 2), (2, 2), (2, 2), (4, 4), (4, 4), (4, 2)], dropout_rate=0.3,
                                    #n_cov_layers=5, n_recurrent_layers=2, n_time_layers=1, n_time_dst_nodes=128,
                                    #n_dense_nodes=240, l2=.001, input_shape=get_input_shape(),
                                    #output_shape=get_output_shape())
#crnn_parallel = crnn_parallel(crnn_parallel_param_grid)
crnn_parallel_fitted, crnn_parallel_history = train_crnn_parallel()
#crnn_weights_path = '../input/crnn-parallel/weights.best.h5'
#crnn_parallel.load_weights(crnn_weights_path)

Train on 6394 samples, validate on 800 samples
Epoch 1/50
6394/6394 [==============================] - 61s 10ms/step - loss: 2.5431 - acc: 0.1950 - val_loss: 2.0438 - val_acc: 0.2313
Epoch 2/50
6394/6394 [==============================] - 58s 9ms/step - loss: 2.1225 - acc: 0.2687 - val_loss: 2.2784 - val_acc: 0.1487
Epoch 3/50
6394/6394 [==============================] - 57s 9ms/step - loss: 1.9294 - acc: 0.3036 - val_loss: 2.0405 - val_acc: 0.1775
Epoch 4/50
6394/6394 [==============================] - 57s 9ms/step - loss: 1.7896 - acc: 0.3427 - val_loss: 2.0590 - val_acc: 0.1838
Epoch 5/50
6394/6394 [==============================] - 54s 8ms/step - loss: 1.7600 - acc: 0.3536 - val_loss: 1.8475 - val_acc: 0.2938
Epoch 6/50
6394/6394 [==============================] - 53s 8ms/step - loss: 1.7010 - acc: 0.3811 - val_loss: 1.9571 - val_acc: 0.2375
Epoch 7/50
6394/6394 [==============================] - 54s 8ms/step - loss: 1.6563 - acc: 0.3929 - val_loss: 1.7686 - val_acc: 0.2988
Epoch 8

In [11]:
train_pred_df, val_pred_df, test_pred_df, crnn_f1_scores = predict_model(crnn_parallel_fitted, 'CRNN Parallel')

800/800 [==============================] - 5s 6ms/step
Training set Accuracy:  0.5652173912857039
Validation set Accuracy:  0.4725
Test set Accuracy:  0.36625


In [12]:
crnn_f1_scores

Training      0.544224
Validation    0.441026
Testing       0.336959
Name: CRNN Parallel, dtype: float64